In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd

# Load the dataset
data = pd.read_csv(r'/Users/kritinreddy/Documents/Capstone final/data_moods.csv')

# Drop duplicates and handle any missing values
data.drop_duplicates(inplace=True)
data.dropna(inplace=True)

# Normalize audio feature columns for better performance in the model
scaler = MinMaxScaler()
audio_features = ['danceability', 'acousticness', 'energy', 'instrumentalness',
                  'liveness', 'valence', 'loudness', 'speechiness', 'tempo']
data[audio_features] = scaler.fit_transform(data[audio_features])

# Split data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

data.head

<bound method NDFrame.head of                                             name  \
0                                           1999   
1                                             23   
2                                       9 Crimes   
3                                 99 Luftballons   
4    A Boy Brushed Red Living In Black And White   
..                                           ...   
681                                  windcatcher   
682              yellow is the color of her eyes   
683                           you broke me first   
684                          you were good to me   
685                                         æfre   

                               album          artist                      id  \
0                               1999          Prince  2H7PHVdQ3mXqEHXcvclTB0   
1                                 23  Blonde Redhead  4HIwL9ii9CcXpTOTzMq0MP   
2                                  9     Damien Rice  5GZEeowhvSieFDiR8fQ2im   
3                     99 

In [3]:
from transformers import Trainer, TrainingArguments, BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset
import torch

# Prepare BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class MusicMoodDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
        # Dynamically create a label map based on all unique moods in the dataset
        self.label_map = {mood: idx for idx, mood in enumerate(data['mood'].unique())}
        print("Label map:", self.label_map)  # Verify the label map includes all moods, including 'Calm'

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        song = self.data.iloc[idx]
        text = f"{song['name']} {song['artist']} {song['album']}"
        inputs = self.tokenizer(
            text, max_length=self.max_len, padding='max_length', truncation=True, return_tensors="pt"
        )
        
        # Use the dynamically created label map
        label = self.label_map[song['mood']]
        
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Assuming train_data and test_data are your DataFrames split from the original dataset
train_dataset = MusicMoodDataset(train_data, tokenizer)
test_dataset = MusicMoodDataset(test_data, tokenizer)

# Load the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased', num_labels=len(train_dataset.label_map)
)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    evaluation_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()

model.save_pretrained(r'/Users/kritinreddy/Documents/Capstone final/fine_tuned_bert_model')
tokenizer.save_pretrained(r'/Users/kritinreddy/Documents/Capstone final/fine_tuned_bert_model')



Label map: {'Sad': 0, 'Energetic': 1, 'Happy': 2, 'Calm': 3}
Label map: {'Calm': 0, 'Happy': 1, 'Energetic': 2, 'Sad': 3}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/anaconda3/lib/python3.12/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/411 [00:00<?, ?it/s]

{'loss': 1.5271, 'grad_norm': 16.291154861450195, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.07}
{'loss': 1.446, 'grad_norm': 12.647690773010254, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.15}
{'loss': 1.4152, 'grad_norm': 9.876943588256836, 'learning_rate': 3e-06, 'epoch': 0.22}
{'loss': 1.3924, 'grad_norm': 10.869108200073242, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.29}
{'loss': 1.4087, 'grad_norm': 10.076031684875488, 'learning_rate': 5e-06, 'epoch': 0.36}
{'loss': 1.3868, 'grad_norm': 8.136679649353027, 'learning_rate': 6e-06, 'epoch': 0.44}
{'loss': 1.3642, 'grad_norm': 9.441511154174805, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.51}
{'loss': 1.3461, 'grad_norm': 8.215226173400879, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.58}
{'loss': 1.3868, 'grad_norm': 9.961832046508789, 'learning_rate': 9e-06, 'epoch': 0.66}
{'loss': 1.3945, 'grad_norm': 11.797051429748535, 'learning_rate': 1e-05, 'epoch': 0.73}
{'loss': 1.325, 'grad_norm': 15.61

  0%|          | 0/35 [00:00<?, ?it/s]

...

  0%|          | 0/35 [00:00<?, ?it/s]

{'eval_loss': 1.572227954864502, 'eval_runtime': 2.8239, 'eval_samples_per_second': 48.869, 'eval_steps_per_second': 12.394, 'epoch': 2.0}
{'loss': 0.9193, 'grad_norm': 19.358598709106445, 'learning_rate': 2.8000000000000003e-05, 'epoch': 2.04}
{'loss': 0.9336, 'grad_norm': 21.089275360107422, 'learning_rate': 2.9e-05, 'epoch': 2.12}
{'loss': 0.8244, 'grad_norm': 17.910980224609375, 'learning_rate': 3e-05, 'epoch': 2.19}
{'loss': 0.948, 'grad_norm': 41.9771614074707, 'learning_rate': 3.1e-05, 'epoch': 2.26}
{'loss': 0.8165, 'grad_norm': 17.889102935791016, 'learning_rate': 3.2000000000000005e-05, 'epoch': 2.34}
{'loss': 0.8284, 'grad_norm': 17.850282669067383, 'learning_rate': 3.3e-05, 'epoch': 2.41}
{'loss': 0.9716, 'grad_norm': 21.28853988647461, 'learning_rate': 3.4000000000000007e-05, 'epoch': 2.48}
{'loss': 1.0232, 'grad_norm': 20.413251876831055, 'learning_rate': 3.5e-05, 'epoch': 2.55}
{'loss': 0.7732, 'grad_norm': 18.629859924316406, 'learning_rate': 3.6e-05, 'epoch': 2.63}
{'l

  0%|          | 0/35 [00:00<?, ?it/s]

{'eval_loss': 1.9228721857070923, 'eval_runtime': 3.1635, 'eval_samples_per_second': 43.622, 'eval_steps_per_second': 11.064, 'epoch': 3.0}
{'train_runtime': 221.7859, 'train_samples_per_second': 7.413, 'train_steps_per_second': 1.853, 'train_loss': 1.1512683438558649, 'epoch': 3.0}


('/Users/kritinreddy/Documents/Capstone final/fine_tuned_bert_model/tokenizer_config.json',
 '/Users/kritinreddy/Documents/Capstone final/fine_tuned_bert_model/special_tokens_map.json',
 '/Users/kritinreddy/Documents/Capstone final/fine_tuned_bert_model/vocab.txt',
 '/Users/kritinreddy/Documents/Capstone final/fine_tuned_bert_model/added_tokens.json')

In [4]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np

# Load the fine-tuned model and tokenizer
model_path = r'/Users/kritinreddy/Documents/Capstone final/fine_tuned_bert_model'

# Load the model (use BertModel instead of BertForSequenceClassification)
model = BertModel.from_pretrained(model_path)

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained(model_path)

# Check if the model and tokenizer are loaded correctly
print(f"Model loaded: {model}")
print(f"Tokenizer loaded: {tokenizer}")

# Function to generate embeddings for all songs
def get_song_embeddings(dataset, tokenizer, model):
    model.eval()
    embeddings = []
    for _, song in dataset.iterrows():
        text = song['name'] + " " + song['artist'] + " " + song['album']
        inputs = tokenizer(text, max_length=128, padding='max_length', truncation=True, return_tensors="pt")

        # Get hidden states from BERT
        with torch.no_grad():
            outputs = model(**inputs)

        # Average the hidden states of the last layer to create the embedding
        # The last hidden state is in 'outputs.last_hidden_state' for BertModel
        song_embedding = outputs.last_hidden_state.mean(dim=1).numpy()
        embeddings.append(song_embedding)

    return np.vstack(embeddings)

# Generate embeddings for all songs
song_embeddings = get_song_embeddings(data, tokenizer, model)


Model loaded: BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1,

In [9]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load the fine-tuned model and tokenizer
model_path = r'/Users/kritinreddy/Documents/Capstone final/fine_tuned_bert_model'

# Load the model (use BertModel instead of BertForSequenceClassification)
model = BertModel.from_pretrained(model_path)

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained(model_path)

# Step 1: Define example sentences to represent each mood
mood_examples = {
    "Happy": [
        "I feel great today!", "I'm so happy and excited!", "This is a wonderful day!",
        "Everything is going so well!", "I'm having the best day ever!", "I can't stop smiling!",
        "I'm in a great mood!", "Feeling so positive and happy!", "Today is a day to celebrate!",
        "I feel on top of the world!", "Everything is working out for me!", "I'm thrilled about life!",
        "I feel so blessed!", "My heart is full of joy!", "I'm loving this moment!",
        "I just received great news!", "The sun is shining, and so am I!", "Life feels so good right now!",
        "I'm radiating happiness!", "This is a happy place!", "I love how today is going!",
        "I woke up feeling great!", "I'm smiling so much today!", "Everything seems to be falling into place.",
        "I love this vibe!", "Feeling so grateful right now!", "Today is the best!", "I’m feeling amazing.",
        "This day is full of positive energy.", "I’m so glad to be here.", "I’m feeling really joyful today!",
        "I can’t stop laughing!", "I’m so proud of how far I’ve come.", "I’m grateful for all of this.",
        "Today is the best day ever!", "My happiness knows no bounds.", "I just can't stop smiling today.",
        "I feel like dancing!", "I’m filled with positivity.", "I feel like I can achieve anything!",
        "My heart is full of joy!", "Everything is going according to plan.", "I’ve never felt better.",
        "I feel like the luckiest person in the world.", "I’m so excited for what’s to come.",
        "I’m bursting with excitement!", "The day is just full of good vibes.", "I feel unstoppable.",
        "I’m on top of the world today.", "I’m so happy I could jump for joy.", "Everything feels right today.",
        "I’m just so happy right now.", "I’m living my best life today!", "I am so thrilled!",
        "What a joyful moment!", "I feel so lucky and happy.", "I’m incredibly happy today!",
        "I’m feeling more energized than ever!", "Everything just feels right in the world.",
        "I’ve never been so happy!", "Today is filled with so much joy.", "I am in such a great mood!",
        "I feel like today is going to be amazing!", "I'm excited about what's next.", "I feel fantastic!",
        "Everything is going better than expected!", "I feel on cloud nine!", "I feel so full of life!",
        "I can’t help but feel happy right now.", "I’m looking forward to everything today.", "I'm so content!",
        "I’m so happy I’m speechless.", "I feel really blessed right now.", "I’m grinning from ear to ear!",
        "I’m in heaven right now.", "I'm feeling blessed.", "Life is just perfect right now!", "This is the life!",
        "I’ve never felt so grateful.", "I feel like the happiest person alive!", "I’m so lucky today!",
        "I'm having the best time of my life!", "I am so fortunate right now.", "Feeling happy and content.",
        "This is such a positive experience.", "I’m on top of the world!", "I love how everything is going.",
        "I'm just so happy with everything right now.", "The world feels right today!", "I feel like I can do anything!"
    ],
    "Sad": [
        "I'm feeling down.", "Today is not my day.", "I feel so sad.", "Everything feels so heavy.",
        "I'm just not feeling it today.", "I can't shake this sadness.", "I don't have the energy for anything.",
        "I'm having a tough time today.", "My heart feels heavy.", "I feel empty inside.",
        "Everything seems so dull right now.", "I just want to be alone.", "Nothing feels right today.",
        "I don't know what's wrong with me.", "I'm overwhelmed by sadness.", "I can't stop thinking about it.",
        "I feel like everything is falling apart.", "I'm not in a good place mentally.", "I feel stuck.",
        "I just want to escape.", "It's hard to be positive right now.", "I feel like crying.",
        "I'm feeling emotionally drained.", "I can't find joy in anything.", "I feel so lost.",
        "Nothing seems to make me happy.", "I can't focus on anything right now.", "I feel completely hopeless.",
        "I don't know what’s going on with me.", "I feel disconnected from everything.", "I feel so lonely.",
        "I'm struggling to keep it together.", "I feel like I’m sinking.", "I can't stop feeling sad.",
        "I feel like I'm drowning in my emotions.", "I don’t want to talk to anyone right now.", "Everything seems pointless.",
        "I'm too tired to care about anything.", "I feel emotionally exhausted.", "I'm just going through the motions.",
        "I feel numb to everything.", "I wish things were different.", "I feel broken inside.",
        "I just want everything to stop.", "I feel like I’m fading away.", "Nothing feels good anymore.",
        "I can't see the light at the end of the tunnel.", "I feel like giving up.", "I feel like I'm not enough.",
        "I'm in a bad mental space.", "I don't feel motivated at all.", "I can’t seem to shake this feeling.",
        "I feel like there’s no way out.", "I’m not looking forward to anything.", "I feel like everything’s too much.",
        "I’m losing hope.", "I feel so disconnected.", "I just want to be left alone.", "Everything feels too overwhelming.",
        "I can’t stop thinking about what went wrong.", "I feel out of place.", "I feel so misunderstood.",
        "I feel like no one cares.", "I can’t seem to find happiness anywhere.", "I feel like I’ve lost everything.",
        "I just want to cry and let it all out.", "I feel drained and empty inside.", "I feel like I’m stuck in a hole.",
        "I feel hopeless and tired.", "I can’t escape these feelings.", "I’m tired of pretending to be okay.",
        "I feel like I’m not in control of my emotions.", "I feel defeated.", "Everything seems to be falling apart.",
        "I’m so tired of everything.", "I feel completely alone in this.", "I don’t know how to make it stop.",
        "I feel emotionally broken.", "I can’t stop crying.", "I'm feeling lost and hopeless.", "Nothing seems to help."
    ],
    "Energetic": [
        "I am enthusiastic!", "Let's get things done!", "I'm feeling very energetic today.",
        "I'm full of energy and ready to go!", "Nothing can stop me today!", "I feel unstoppable!",
        "I have so much energy right now!", "I'm ready to take on the world!", "I'm pumped up!",
        "Let's make today amazing!", "I’m on fire today!", "I feel like I could conquer anything.",
        "Let's get moving!", "I'm bursting with energy!", "I can't sit still today!",
        "I’m all charged up!", "I'm ready to take on whatever comes my way!", "I’ve got so much motivation!",
        "I'm full of life and energy!", "Let's make it happen today!", "I feel like a powerhouse!",
        "I’m feeling energized!", "I’m ready to run a marathon!", "I feel like a superhero today!",
        "I’m firing on all cylinders!", "Nothing is slowing me down today.", "I feel totally alive!",
        "I’m ready to tackle anything that comes my way!", "My energy levels are through the roof!",
        "I feel like I can take on the world!", "I’ve got this in the bag!", "I’m charged up and ready to go!",
        "I’m so hyped up!", "I feel like I’ve got endless energy today.", "Let’s get this party started!",
        "I feel like dancing!", "I’m ready for anything!", "I feel so motivated to succeed!",
        "Today’s going to be a productive day!", "I’m filled with energy and creativity.", "Nothing can stop me now!",
        "I feel so driven!", "Let’s do this!", "I’m ready to take on the world!", "I feel so powerful!",
        "I’m ready for whatever comes my way!", "I’m in the mood to work hard!", "I’ve got more energy than I know what to do with!",
        "I can’t stop moving today!", "I’m full of ideas!", "I’m feeling the adrenaline!",
        "I feel like taking on new challenges.", "I’m pumped up for success!", "I’m feeling unstoppable today!",
        "I feel like I'm at my best right now.", "I’m excited to see where today takes me.", "I’m totally ready to go.",
        "I’m ready for anything the day throws at me.", "I'm moving at full speed today!", "I feel so charged up!",
        "I'm ready to crush my goals!", "Nothing can hold me back today!", "I’m so motivated and ready to win!",
        "I feel like I'm in my prime.", "I’m at the top of my game today!", "I can feel the energy flowing through me.",
        "I’m ready to take on the world with full force!", "I’m super motivated!"],
    "Calm": [
        "I'm calm and peaceful.", "Feeling very relaxed.", "It's a nice, quiet day.",
        "I feel centered and at peace.", "I’m in a tranquil state of mind.", "Everything feels serene.",
        "I’m just enjoying this peaceful moment.", "I feel very at ease.", "I’m in my zen place.",
        "It’s so quiet and calming.", "I feel grounded.", "I’m in a state of mindfulness.",
        "I feel balanced and calm.", "Everything is so still right now.", "I’m just breathing deeply and relaxing.",
        "I feel totally at peace.", "I’m taking it easy today.", "There’s a calmness in the air.",
        "I’m not rushing today, just going with the flow.", "I feel like everything is in perfect harmony.",
        "I’m enjoying the stillness around me.", "Everything is calm and steady.", "I’m feeling calm and collected.",
        "I’m enjoying the quiet moments.", "I’m in a peaceful state of mind.", "I’m at peace with myself.",
        "I’m feeling really grounded today.", "I’m completely relaxed.", "The day feels so serene.",
        "I’m in the mood for some relaxation.", "I feel at ease with everything around me.", "Everything seems so peaceful.",
        "I feel calm in the midst of chaos.", "I’m feeling the stillness around me.", "I feel a deep sense of calm.",
        "I’m not stressing about anything today.", "I’m just enjoying the calmness of the moment.", "There’s a gentle calmness in the air.",
        "I feel like everything is in its right place.", "I’m feeling light and unburdened.", "I’m taking a moment for myself.",
        "The world feels at peace around me.", "I’m feeling calm and tranquil.", "I’m in no rush today.",
        "I’m savoring the peace and quiet.", "I’m embracing this quiet moment.", "I’m enjoying the serenity around me.",
        "Everything feels so effortless today.", "I’m enjoying the calm energy in the air.", "I’m feeling calm and content.",
        "I’m enjoying this peaceful atmosphere.", "I’m at ease with everything happening around me.", "I feel so relaxed and balanced.",
        "I’m embracing the calmness of this moment.", "I’m not letting anything disturb my peace.", "Everything feels light and peaceful.",
        "I’m feeling so serene right now.", "I’m feeling completely at ease.", "I’m enjoying this quiet time.",
        "I’m feeling so relaxed and calm.", "I feel like I’m floating on air.", "I’m in a peaceful, relaxed state.",
        "There’s a soothing calmness around me.", "I’m feeling calm and unhurried.", "Everything is moving at the perfect pace.",
        "I’m really at peace with the situation.", "I’m soaking in the tranquility of the moment.", "I’m feeling calm and clear-headed.",
        "I’m in a slow and peaceful mood today.", "I’m at ease with my surroundings.", "The stillness is so calming.",
        "I’m feeling the serenity of the moment.", "I’m enjoying the calm flow of things.", "I feel completely at peace with myself.",
        "Everything feels balanced and peaceful.", "I’m in the mood for some quiet reflection.", "I’m letting everything unfold naturally.",
        "I’m feeling calm despite everything happening around me.", "I’m savoring the quiet moments today.", "I feel the peace within me.",
        "I’m embracing the stillness and quiet.", "I feel so relaxed and at peace with my surroundings.", "I’m content with the way things are going.",
        "I’m enjoying the calm of nature today.", "I feel like everything is in a peaceful rhythm.", "I’m in the mood to be still and quiet.",
        "I’m feeling so centered today.", "Everything feels perfect just the way it is.", "I feel calm even when things are busy.",
        "I’m taking things one step at a time.", "I’m enjoying this quiet, peaceful moment.", "I feel a deep sense of tranquility today.",
        "I’m feeling peaceful and calm inside.", "I’m at peace with everything happening in my life.", "I’m finding peace in the little things.",
        "I’m feeling so composed and centered.", "Everything feels so simple and peaceful.", "I’m feeling calm and untroubled today.",
        "I’m at peace with where I am in life.", "I feel like I’m in a peaceful bubble.", "I’m letting go of all the tension.",
        "I’m in a peaceful, relaxed state of mind.", "I’m feeling calm and in control of my emotions.", "I’m allowing myself to relax and unwind.",
        "I’m embracing the peaceful flow of today.", "I’m feeling calm and at ease with my thoughts.", "I’m grateful for this peaceful moment.",
        "I’m centered and at ease with everything.", "I’m taking everything in stride today.", "I feel calm and ready for whatever comes.",
        "I’m enjoying this peaceful rhythm in my life.", "I’m feeling calm, even in the midst of chaos.", "I’m at peace with my present situation.",
        "I’m soaking in the peaceful moments of today.", "I’m feeling calm in the face of uncertainty.", "I feel a quiet peace inside me."
    ]
}
# Step 2: Generate embeddings for each mood category based on example sentences
def get_sentence_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)  # Use 'model' here instead of 'bert_model'
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Generate average embeddings for each mood by averaging embeddings of example sentences
mood_embeddings = {}
for mood, examples in mood_examples.items():
    example_embeddings = [get_sentence_embedding(sentence) for sentence in examples]
    mood_embeddings[mood] = np.mean(example_embeddings, axis=0)

# Step 3: Define a function to detect mood based on user input
def detect_mood(user_input):
    user_embedding = get_sentence_embedding(user_input)
    similarities = {mood: cosine_similarity([user_embedding], [embedding])[0][0] for mood, embedding in mood_embeddings.items()}
    detected_mood = max(similarities, key=similarities.get)
    return detected_mood

# Step 4: Recommend songs based on detected mood
def recommend_songs(detected_mood, data, song_embeddings, top_k=5):
    mood_songs = data[data['mood'] == detected_mood]

    if mood_songs.empty:
        print(f"No songs found for the mood: {detected_mood}")
        return []

    mood_indices = mood_songs.index
    mood_embeddings = song_embeddings[mood_indices]

    # Calculate cosine similarity
    similarity_matrix = cosine_similarity(mood_embeddings)

    recommendations = []
    for idx in range(len(mood_indices)):
        similar_indices = similarity_matrix[idx].argsort()[::-1][1:top_k+1]
        similar_songs = mood_songs.iloc[similar_indices]
        recommendations.extend(similar_songs[['name', 'artist', 'album', 'mood']].values.tolist())

    return recommendations

# Step 5: Prompt user for input and recommend songs based on detected mood
user_input = input("How are you feeling today? Describe your mood: ")
detected_mood = detect_mood(user_input)
print(f"Detected Mood: {detected_mood}")

# Recommend songs (assuming song_embeddings and data are already defined)
recommended_songs = recommend_songs(detected_mood, data, song_embeddings)
if recommended_songs:
    print("\nRecommended Songs:")
    for song in recommended_songs:
        print(f"Name: {song[0]}, Artist: {song[1]}, Album: {song[2]}, Mood: {song[3]}")
else:
    print("No recommendations available for the detected mood.")


Detected Mood: Energetic

Recommended Songs:
Name: Reinventing Your Exit, Artist: Underoath, Album: They're Only Chasing Safety, Mood: Energetic
Name: Away From Me, Artist: Puddle Of Mudd, Album: Life On Display, Mood: Energetic
Name: Out Of Control, Artist: Hoobastank, Album: The Reason, Mood: Energetic
Name: Same Direction, Artist: Hoobastank, Album: The Reason, Mood: Energetic
Name: There For You, Artist: Gorgon City, Album: There For You, Mood: Energetic
Name: Papercut, Artist: Linkin Park, Album: Hybrid Theory (Bonus Edition), Mood: Energetic
Name: One Step Closer, Artist: Linkin Park, Album: Hybrid Theory, Mood: Energetic
Name: Count That - R3HAB Edit, Artist: Steff da Campo, Album: Count That (R3HAB Edit), Mood: Energetic
Name: Points of Authority, Artist: Linkin Park, Album: Hybrid Theory, Mood: Energetic
Name: Killing In The Name, Artist: Rage Against The Machine, Album: Rage Against The Machine - XX (20th Anniversary Special Edition), Mood: Energetic
Name: From Yesterday, Art